## Prerequisites

# 파이썬을 아나콘다로 설치하신 분들은,
# 아래의 코드를 실행하면 LightGBM을 설치할 수 있습니다.
#!conda install -c -y conda-forge lightgbm

In [1]:
# 파이썬을 아나콘다로 설치하지 않은 분들은(그리고 pip가 설치되어 있는 분들은),
# 아래의 코드를 실행하면 LightGBM을 설치할 수 있습니다.
#!pip install lightgbm

In [2]:
# 아래 코드를 실행해보세요.
# 만일 실행이 되지 않는다면 다음의 링크 https://goo.gl/w45GHy 의 내용을 바탕으로 LightGBM을 재설치 해주세요.
import lightgbm

In [3]:
random_state = 42
random_state

42

## Otto Group Product Classification Challenge

이번 과제는 세계 최대의 전자상거래 회사 중 하나인 [Otto Group](https://www.ottogroup.com/)에서 주최하는 [Otto Group Product Classification Challenge](https://www.kaggle.com/c/otto-group-product-classification-challenge/) 경진대회에 참석해보겠습니다.

Otto Group은 익명화(anonymization)된 상품 정보에 대한 데이터를 제공하는데, 경진대회 참석자는 이 데이터를 활용하여 주어진 상품 카테고리(target)를 예측해야 합니다. 상품 카테고리는 Class_1부터 Class_9까지 총 9개가 있습니다. 주어진 데이터를 Decision Tree, Random Forest, 그리고 Gradient Boosting Machine를 활용하여 예측해보도록 하겠습니다.




In [4]:
import pandas as pd

data = pd.read_csv("data/train.csv", index_col="id")

print(data.shape)
data.head()

(61878, 94)


,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
id,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
4,1,0,0,1,6,1,5,0,0,1,...,0,1,2,0,0,0,0,0,0,Class_1
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


## Preprocessing

In [5]:
label_name = "target"
label_name

'target'

In [6]:
feature_names = data.columns.difference([label_name])

print(len(feature_names))
feature_names

93


Index(['feat_1', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14',
       'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_2',
       'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25',
       'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_3', 'feat_30',
       'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36',
       'feat_37', 'feat_38', 'feat_39', 'feat_4', 'feat_40', 'feat_41',
       'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47',
       'feat_48', 'feat_49', 'feat_5', 'feat_50', 'feat_51', 'feat_52',
       'feat_53', 'feat_54', 'feat_55', 'feat_56', 'feat_57', 'feat_58',
       'feat_59', 'feat_6', 'feat_60', 'feat_61', 'feat_62', 'feat_63',
       'feat_64', 'feat_65', 'feat_66', 'feat_67', 'feat_68', 'feat_69',
       'feat_7', 'feat_70', 'feat_71', 'feat_72', 'feat_73', 'feat_74',
       'feat_75', 'feat_76', 'feat_77', 'feat_78', 'feat_79', 'feat_8',
       'feat_80', 'feat_81', 'feat_82', 'feat_83', 'feat_84

In [7]:
X = data[feature_names]

print(X.shape)
X.head()

(61878, 93)


,feat_1,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_9,feat_90,feat_91,feat_92,feat_93
id,,,,,,,,,,,,,,,,,,,,,
1,1,0,1,0,0,0,0,0,2,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,1,0,0,1,1,0,...,0,1,2,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,4,0,...,1,0,0,0,0,0,1,0,0,0


In [8]:
y = data[label_name]

print(y.unique())

print(y.shape)
y.head()

['Class_1' 'Class_2' 'Class_3' 'Class_4' 'Class_5' 'Class_6' 'Class_7'
 'Class_8' 'Class_9']
(61878,)


id
1    Class_1
2    Class_1
3    Class_1
4    Class_1
5    Class_1
Name: target, dtype: object

### Benchmark

데이터를 읽어왔다면 이제 Decision Tree, Random Forest, 그리고 Gradient Boosting Machine으로 상품의 카테고리를 예측해보겠습니다. 먼저 Decision Tree부터 사용하겠습니다.

### DecisionTree

In [10]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42)
model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [11]:
model.fit(X, y)
y_predict = model.predict(X)

print(y_predict.shape)
y_predict[0:5]

(61878,)


array(['Class_1', 'Class_1', 'Class_1', 'Class_1', 'Class_1'],
      dtype=object)

In [17]:
score = (y_predict == y).mean()

# Overfitting!!
print(f"Score = {score:.6f}")

Score = 1.000000


### Hold-Out Validation

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(43314, 93) (43314,)
(18564, 93) (18564,)


In [24]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

y_train_predict = model.predict(X_train)
y_test_predict = model.predict(X_test)

train_score = (y_train_predict == y_train).mean()
test_score = (y_test_predict == y_test).mean()

print(f"Accuracy(train) = {train_score:.6f}")
print(f"Accuracy(test) = {test_score:.6f}")

Accuracy(train) = 1.000000
Accuracy(test) = 0.709869


### Hyperparameter

In [22]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=20, random_state=42)
model.fit(X_train, y_train)

y_train_predict = model.predict(X_train)
y_test_predict = model.predict(X_test)

train_score = (y_train_predict == y_train).mean()
test_score = (y_test_predict == y_test).mean()

print(f"Accuracy(train) = {train_score:.6f}")
print(f"Accuracy(test) = {test_score:.6f}")

Accuracy(train) = 0.824445
Accuracy(test) = 0.717787


### Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_train_predict = model.predict(X_train)
y_test_predict = model.predict(X_test)

train_score = (y_train_predict == y_train).mean()
test_score = (y_test_predict == y_test).mean()

print(f"Accuracy(train) = {train_score:.6f}")
print(f"Accuracy(test) = {test_score:.6f}")

Accuracy(train) = 0.993028
Accuracy(test) = 0.779412


### Gradient Boosting Machine(LIghtGBM)

In [30]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=42)
model.fit(X_train, y_train)

y_train_predict = model.predict(X_train)
y_test_predict = model.predict(X_test)

train_score = (y_train_predict == y_train).mean()
test_score = (y_test_predict == y_test).mean()

print(f"Accuracy(train) = {train_score:.6f}")
print(f"Accuracy(test) = {test_score:.6f}")

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy(train) = 0.854135
Accuracy(test) = 0.808285


C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
